In [5]:

from utils.logging_utils import create_logger
from yaml import safe_load
from pathlib import Path
from ingestion.database import VectorDataBase, PostgresDatabase
from utils.logging_utils import create_logger
from preprocessing.spacy_prep import SpacyPrep
from modeling_clusterization.embedding import FastTextModel
from modeling_clusterization.gpt_models import ChatGpt, Gpt4All
import os



In [7]:

# with open('src/universal_config.yaml', 'r') as file:
with open('universal_config.yaml', 'r') as file:

    cf = safe_load(file)

cf_db_p = cf['database']['postgres']
cf_db_q = cf['database']['qdrant']


# with open('src/application_questions_config.yaml', 'r') as file:
with open('application_questions_config.yaml', 'r') as file:

    cf_appq = safe_load(file)


# parent_folder_path = Path().cwd()
parent_folder_path = Path(cf['parent_path'])
print(parent_folder_path)
data_path = parent_folder_path.parent / 'data' 
app_questions_data_path = data_path / 'app_questions' 
embed_model_path = app_questions_data_path / 'artifacts' / 'fasttext.model'
fasttext_training_it_dataset_path = app_questions_data_path / 'prep_data' / 'prep_sentences_50K.csv'


# paths to gpt binaries and generated output csv files
cf_gpt_model_to_use = cf_appq['gpt_model_constants']['gpt_model_to_use']
model_path = parent_folder_path.parent / 'data' / cf['gpt_model_constants']['model_path']
# model_path = parent_folder_path / 'data' / cf['gpt_model_constants']['model_path']  ## for .py file
gpt_model_str =  cf['gpt_model_constants'][cf_gpt_model_to_use]['gpt_model_str'] #or use gpt4all constants

# strings to generate prompts
system_string = cf_appq['gpt_model_constants']['default_prompt_strings']['system_string']
user_string = cf_appq['gpt_model_constants']['default_prompt_strings']['user_string']
substring_to_replace = cf_appq['gpt_model_constants']['default_prompt_strings']['substring_to_replace']
different_answer_user_string = cf_appq['gpt_model_constants']['default_prompt_strings']['different_answer_user_string']

# other gpt constants  - use .env variables, so that these can be altered easily
hist_qa_return_limit = int(os.getenv('HIST_QA_RETURN_LIMIT'))
gpt_answer_return_limit = int(os.getenv('GPT_ANSWER_RETURN_LIMIT'))
 

# logging constants and pipeline logger
log_level = cf['utils']['log_level']
tpb_logger = create_logger(log_level, log_name = 'application_questions_pipeline_log')
tpb_logger.info('running application_questions_pipeline.py')

postgres_db = PostgresDatabase()
spc = SpacyPrep()
embed_model = FastTextModel(embed_model_path)
qdrant_db = VectorDataBase( cf_db_q['questions']['table_name'],
                                cf_db_q['vec_size'])

# if cf_gpt_model_to_use == 'chatgpt':
#     chatgpt_api_timing_delay =  cf['gpt_model_constants'][cf_gpt_model_to_use]['chatgpt_api_timing_delay']
#     gpt_model = ChatGpt(gpt_model_str, model_path, chatgpt_api_timing_delay, log_level)
# else:
#     gpt_model = Gpt4All(gpt_model_str, model_path, log_level)


[08-09 01:16:18] {application_questions_pipeline_log:45} INFO - running application_questions_pipeline.py


src


In [8]:
Path().cwd()

WindowsPath('d:/project/ai_job_autopilot/gitlab/ai_core/src')

In [3]:
from sessions.application_session import ApplicationSession


app_session = ApplicationSession(postgres_db, spc, 
                embed_model, qdrant_db,
                gpt_model, fasttext_training_it_dataset_path,
                log_level)
app_session.initialize_session()
hist_answer_question_dict, ret_list = app_session.query_tune_answer(user_string, system_string, substring_to_replace, 
                        different_answer_user_string, new_question = 'Do you have experience in customer service?',
                        hist_qa_return_limit=3, gpt_answer_return_limit = 3
                        )



[08-09 00:29:42] {PostgresDatabase:259} INFO - Postgres database core_backend_stage_db has been connected 
[08-09 00:29:42] {VectorDataBase:75} INFO - Qdrant database has been connected
[08-09 00:29:46] {FastTextModel:93} INFO - Loaded fast text model from disk
[08-09 00:29:46] {application_session.py:51} INFO - Application session initialized
[08-09 00:29:46] {SpacyPrep:86} INFO - cleaning words to a list words
[08-09 00:29:46] {SpacyPrep:104} INFO - cleaning list of sentences to a list of lists of words
[08-09 00:29:46] {FastTextModel:131} INFO - Converting strings of text to vectors via fasttext model
[08-09 00:29:46] {VectorDataBase:167} INFO - Qdrant table questions has been queried
d:\project\ai_job_autopilot\gitlab\ai_core\src\ingestion\database.py:392: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  self.df_a = psql.read_sql(

Found model file at  d:\\\\project\\\\ai_job_autopilot\\\\gitlab\\\\ai_core\\\\data\\\\job_posts\\\\artifacts\\\\gpt_models\\ggml-gpt4all-j-v1.3-groovy.bin


[08-09 00:30:27] {gpt_models-gpt4All_log:65} INFO - attempting to generate gpt4all predictions



Found model file at  d:\\\\project\\\\ai_job_autopilot\\\\gitlab\\\\ai_core\\\\data\\\\job_posts\\\\artifacts\\\\gpt_models\\ggml-gpt4all-j-v1.3-groovy.bin


[08-09 00:31:04] {gpt_models-gpt4All_log:65} INFO - attempting to generate gpt4all predictions


...

[08-09 00:31:51] {gpt_models-gpt4All_log:65} INFO - attempting to generate gpt4all predictions


[08-09 00:31:52] {application_session.py:106} INFO - GPT model generated a tuned answer
[08-09 00:31:52] {application_session.py:107} INFO - Answer:  I have experience in customer service, and my previous role at XYZ Retail involved interacting with customers daily, handling transactions and ensuring a positive shopping experience.


Found model file at  d:\\\\project\\\\ai_job_autopilot\\\\gitlab\\\\ai_core\\\\data\\\\job_posts\\\\artifacts\\\\gpt_models\\ggml-gpt4all-j-v1.3-groovy.bin


[08-09 00:32:45] {gpt_models-gpt4All_log:65} INFO - attempting to generate gpt4all predictions


[08-09 00:32:46] {application_session.py:106} INFO - GPT model generated a tuned answer
[08-09 00:32:46] {application_session.py:107} INFO - Answer:  I have experience in customer service, and my previous role at XYZ Retail involved interacting with customers daily, handling transactions and ensuring a positive shopping experience.


NameError: name 'hist_answer_question_dict' is not defined